In [250]:
from google.colab import drive 
drive.mount('/content/gdrive')
import pandas as pd
import numpy as np
import re 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [305]:
#!pip install transformers
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [251]:
!pip install datasets evaluate seqeval
!pip install bioc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [252]:
data = pd.read_csv("gdrive/Shared drives/capstone/TestPapers.csv")
data = data[['Type','Website','text/abstract','Labels']]
data.head()

,Type,Website,text/abstract,Labels
0,article,https://www.sciencenews.org/article/hibernatin...,"Heat shock protein 47, or HSP47, is normally f...","Labels:[(""Heat shock protein 47"", ""GeneOrGeneP..."
1,article,https://www.fiercebiotech.com/research/new-tes...,"SYNTap is a seed amplification assay, or SAA, ...","Labels:[(""Creutzfeldt-Jakob disease"", ""Disease..."
2,article,https://www.fiercebiotech.com/research/discove...,Professor Adrian Bracken and his team had been...,"Labels:[(""Polycomb protein complexes"", ""GeneOr..."
3,article,https://www.fiercebiotech.com/research/returni...,The breakthrough from researchers including at...,"Labels:[(""neuronal"", ""CellLine""), (""pentraxin ..."
4,article,https://www.fiercebiotech.com/medtech/new-test...,"Currently being explored in cancer testing, DN...","Labels:[(""cancer"", ""DiseaseOrPhenotypicFeature..."


In [253]:
'''labelsnew = [{"Heat shock protein 47":"GeneOrGeneProduct", "HSP47":"GeneOrGeneProduct", "platelets":"CellLine", "collagen":"GeneOrGeneProduct", "blood clots":"DiseaseOrPhenotypicFeature"},
{"Creutzfeldt-Jakob disease":"DiseaseOrPhenotypicFeature", "Parkinson's":"DiseaseOrPhenotypicFeature", "alpha-synuclein":"GeneOrGeneProduct", "Lewy bodies":"DiseaseOrPhenotypicFeature", "neurons":"CellLine", "dopamine":"ChemicalEntity", "tremors":"DiseaseOrPhenotypicFeature", "Alzheimer’s":"DiseaseOrPhenotypicFeature", "LRRK2":"GeneOrGeneProduct"},
{"Polycomb protein complexes":"GeneOrGeneProduct", "PRC1":"GeneOrGeneProduct", "PRC2":"GeneOrGeneProduct", "neuron cell":"CellLine", "PRC2.1":"SequenceVariant", "PRC2.2":"SequenceVariant"},
{"neuronal":"CellLine", "pentraxin protein":"GeneOrGeneProduct", "Nptx2":"GeneOrGeneProduct", "excitatory neurons":"CellLine", "Alzheimer’s disease":"DiseaseOrPhenotypicFeature", "frontotemporal dementia":"DiseaseOrPhenotypicFeature", "schizophrenia":"DiseaseOrPhenotypicFeature", "Down syndrome":"DiseaseOrPhenotypicFeature"},
{"cancer":"DiseaseOrPhenotypicFeature", "diabetes":"DiseaseOrPhenotypicFeature", "Type 2 diabetes":"DiseaseOrPhenotypicFeature"},
{"Venous thromboembolism":"DiseaseOrPhenotypicFeature", "VTE":"DiseaseOrPhenotypicFeature", "deep venous thrombosis":"DiseaseOrPhenotypicFeature", "pulmonary embolism":"DiseaseOrPhenotypicFeature", "spinal cord injury":"DiseaseOrPhenotypicFeature", "SCI":"DiseaseOrPhenotypicFeature", "heat shock protein 47":"GeneOrGeneProduct", "HSP47":"GeneOrGeneProduct", "platelets":"CellLine","neutrophil":"CellLine"},
{"Parkinson's disease":"DiseaseOrPhenotypicFeature", "dopaminergic deficit":"DiseaseOrPhenotypicFeature", "prodromal":"DiseaseOrPhenotypicFeature", "non-manifesting carriers":"DiseaseOrPhenotypicFeature", "α-synuclein":"GeneOrGeneProduct", "LRRK2":"GeneOrGeneProduct", "RBD":"DiseaseOrPhenotypicFeature", "hyposmia":"DiseaseOrPhenotypicFeature"},
{"Polycomb repressive complex 2":"GeneOrGeneProduct", "EED":"GeneOrGeneProduct", "SUZ12":"GeneOrGeneProduct", "EZH1/2":"GeneOrGeneProduct", "histone H3":"GeneOrGeneProduct", "PRC2.1":"SequenceVariant", "PCL1–3":"GeneOrGeneProduct", "EPOP":"GeneOrGeneProduct", "PALI1/2":"GeneOrGeneProduct", "PRC2.2":"SequenceVariant", "AEBP2":"GeneOrGeneProduct", "JARID2":"GeneOrGeneProduct", "mouse embryonic stem cells":"CellLine", "Polycomb":"GeneOrGeneProduct", "RING1A/B":"GeneOrGeneProduct", "PCLs":"GeneOrGeneProduct"},
{"microglial":"CellLine", "Alzheimer’s disease":"DiseaseOrPhenotypicFeature", "AD":"DiseaseOrPhenotypicFeature", "frontotemporal dementia":"DiseaseOrPhenotypicFeature", "FTD":"DiseaseOrPhenotypicFeature", "pentraxin":"GeneOrGeneProduct", "Nptx2":"GeneOrGeneProduct", "C1q":"GeneOrGeneProduct", "Nptx2-C1q":"GeneOrGeneProduct", "C3":"GeneOrGeneProduct"},
{"Type 2 diabetes mellitus":"DiseaseOrPhenotypicFeature", "T2D":"DiseaseOrPhenotypicFeature"},
{"pluripotent stem cells":"CellLine", "endothelial cells":"CellLine", "mesodermal cells":"CellLine", "inflammation":"DiseaseOrPhenotypicFeature", "tumor formation":"DiseaseOrPhenotypicFeature", "Down syndrome":"DiseaseOrPhenotypicFeature", "myosin":"GeneOrGeneProduct"},
{"R805":"ChemicalEntity", "CX-011":"ChemicalEntity", "inflammation":"DiseaseOrPhenotypicFeature", "osteoarthritis":"DiseaseOrPhenotypicFeature", "Cartilage erosion":"DiseaseOrPhenotypicFeature", "gp130":"GeneOrGeneProduct"},
{"Down syndrome":"DiseaseOrPhenotypicFeature", "DS":"DiseaseOrPhenotypicFeature", "trisomy 21":"SequenceVariant", "T21":"SequenceVariant", "cognitive impairment":"DiseaseOrPhenotypicFeature", "muscle hypotonia":"DiseaseOrPhenotypicFeature", "heart defects":"DiseaseOrPhenotypicFeature", "solid tumor formation":"DiseaseOrPhenotypicFeature", "iPSC-derived stromal cells":"CellLine", "mesodermal progenitors":"CellLine", "stromal progenitor":"CellLine", "actin":"GeneOrGeneProduct", "myosin":"GeneOrGeneProduct", "Collagens":"GeneOrGeneProduct", "Galectin-1":"GeneOrGeneProduct", "Fibronectin":"GeneOrGeneProduct", "Heparan":"GeneOrGeneProduct", "Sulfate":"ChemicalEntity", "LOX":"GeneOrGeneProduct", "FAK1":"GeneOrGeneProduct", "USP16":"GeneOrGeneProduct", "S1P complexes":"GeneOrGeneProduct", "miR-24-4":"GeneOrGeneProduct", "miR-21":"GeneOrGeneProduct"},
{"interleukin 6":"GeneOrGeneProduct", "IL-6":"GeneOrGeneProduct", "cytokines":"GeneOrGeneProduct", "glycoprotein 130":"GeneOrGeneProduct", "gp130":"GeneOrGeneProduct", "signaling tyrosine 814":"GeneOrGeneProduct", "Y814":"GeneOrGeneProduct", "F814":"GeneOrGeneProduct", "SRC":"GeneOrGeneProduct", "MAPK":"GeneOrGeneProduct", "R805":"ChemicalEntity", "hair follicle neogenesis":"DiseaseOrPhenotypicFeature", "dermal regeneration":"DiseaseOrPhenotypicFeature", "osteoarthritis":"DiseaseOrPhenotypicFeature"},
{"human adenovirus 52-kDa protein":"GeneOrGeneProduct", "capsid":"GeneOrGeneProduct", "adenovirus":"OrganismTaxon"},
{"HDAC":"GeneOrGeneProduct", "LKB1":"GeneOrGeneProduct", "cancer":"DiseaseOrPhenotypicFeature", "tumor":"DiseaseOrPhenotypicFeature", "HDAC3":"GeneOrGeneProduct","NSCLC":"DiseaseOrPhenotypicFeature"},
{"USB1":"GeneOrGeneProduct", "hematopoietic failure":"DiseaseOrPhenotypicFeature", "poikiloderma":"DiseaseOrPhenotypicFeature", "neutropenia":"DiseaseOrPhenotypicFeature", "PN": "DiseaseOrPhenotypicFeature", "U6 small nuclear RNA":"GeneOrGeneProduct", "pre-mRNA":"GeneOrGeneProduct", "c.531_delA":"SequenceVariant", "microRNA":"GeneOrGeneProduct", "miRNA": "GeneOrGeneProduct", "PAPD5/7":"GeneOrGeneProduct", "miRNA 3′-end adenylation":"SequenceVariant", "hematopoiesis":"DiseaseOrPhenotypicFeature", "deadenylase":"GeneOrGeneProduct"},
{"microRNA":"GeneOrGeneProduct", "PAPD5":"GeneOrGeneProduct", "RG7834":"ChemicalEntity", "RO7020322":"ChemicalEntity", "hepatitis B":"DiseaseOrPhenotypicFeature", "UB1":"SequenceVariant", "RG7834":"ChemicalEntity", "microRNA":"GeneOrGeneProduct", "neutrophil":"CellLine", "PN":"DiseaseOrPhenotypicFeature"},
{"CRISPR-Cas13 proteins":"GeneOrGeneProduct", "RNA-guided RNA nucleases":"GeneOrGeneProduct", "phage":"OrganismTaxon", "RNA":"GeneOrGeneProduct", "LbuCas13a":"GeneOrGeneProduct", "Cas13a":"GeneOrGeneProduct"},
{"microglia":"CellLine", "Huntington’s disease":"DiseaseOrPhenotypicFeature", "tau":"GeneOrGeneProduct", "dementia":"DiseaseOrPhenotypicFeature"},
{"insomnia":"DiseaseOrPhenotypicFeature", "restless leg syndrome":"DiseaseOrPhenotypicFeature", "FABP7":"GeneOrGeneProduct", "ZFYVE28":"GeneOrGeneProduct"},
{"Uveitis":"DiseaseOrPhenotypicFeature", "glaucoma":"DiseaseOrPhenotypicFeature", "cancer":"DiseaseOrPhenotypicFeature", "P-TEFb ":"GeneOrGeneProduct", " T helper cells":"CellLine"},
{"otulipenia":"DiseaseOrPhenotypicFeature", "OTULIN":"GeneOrGeneProduct", "ubiquitin":"GeneOrGeneProduct", "rheumatoid arthritis":"DiseaseOrPhenotypicFeature"},
{"Joubert syndrome":"DiseaseOrPhenotypicFeature", "CEP290":"GeneOrGeneProduct", "kidney cells":'CellLine', "kidney disease":"DiseaseOrPhenotypicFeature"},
{"Parkinson’s disease":"DiseaseOrPhenotypicFeature",  "PD":"DiseaseOrPhenotypicFeature", "c-Abl ":"GeneOrGeneProduct", "leukemia":"DiseaseOrPhenotypicFeature", "α-synuclein":"GeneOrGeneProduct"},
{"tauopathies":"DiseaseOrPhenotypicFeature", "Huntington’s disease":"DiseaseOrPhenotypicFeature", "HD":"DiseaseOrPhenotypicFeature", "aggregate-prone proteins":"GeneOrGeneProduct", "neurodegeneration":"DiseaseOrPhenotypicFeature", "Alzheimer’s disease":"DiseaseOrPhenotypicFeature", "microglia":"CellLine", "cytokines":"GeneOrGeneProduct", "chemokines":"GeneOrGeneProduct", "neuronal receptors":"GeneOrGeneProduct", "CCL3":"GeneOrGeneProduct", "CCL4":"GeneOrGeneProduct", "CCL5":"GeneOrGeneProduct", "CCR5":"GeneOrGeneProduct"},
{"insomnia":"DiseaseOrPhenotypicFeature", "restless legs syndrome":"DiseaseOrPhenotypicFeature", "coronary artery disease":"DiseaseOrPhenotypicFeature", },
{"cancer":"DiseaseOrPhenotypicFeature", "chimeric antigen receptor":"GeneOrGeneProduct", "T cells": "CellLine", "refractory B cell leukemia":"DiseaseOrPhenotypicFeature", "CAR Tregs":"GeneOrGeneProduct", "NK cells":"GeneOrGeneProduct"},
{"Otulipenia":"DiseaseOrPhenotypicFeature", "Inflammation":"DiseaseOrPhenotypicFeature", "bacteria":"OrganismTaxon", "viruses":"OrganismTaxon", "OTULIN":"GeneOrGeneProduct", "ubiquitin":"GeneOrGeneProduct", "cytokines":"GeneOrGeneProduct"},
{"renal ciliopathies":"DiseaseOrPhenotypicFeature", "cystic kidney disease":"DiseaseOrPhenotypicFeature", "CEP290":"GeneOrGeneProduct", "end stage renal failure":"DiseaseOrPhenotypicFeature", "ESRF":"DiseaseOrPhenotypicFeature", "CEP290-associated JBTS":"DiseaseOrPhenotypicFeature", "antisense oligonucleotide":"GeneOrGeneProduct", "ASO": "GeneOrGeneProduct", "41, G1890":"SequenceVariant"},
{"c-Abl ":"GeneOrGeneProduct", "Parkinson's disease":"DiseaseOrPhenotypicFeature", "leukemia":"DiseaseOrPhenotypicFeature"},
{"METTL3":"GeneOrGeneProduct", "RNA N6-methyladenosine":"SequenceVariant", "m6A, methyltransferase":"GeneOrGeneProduct", "m6A methylation":"SequenceVariant", "mRNA":"GeneOrGeneProduct", "Alzheimer’s":"DiseaseOrPhenotypicFeature", "myeloid cells":"CellLine", "macrophages":"OrganismTaxon", "microglia":"CellLine"},
{"Hepatocellular carcinoma":"DiseaseOrPhenotypicFeature", "hepatocarcinogenesis":"DiseaseOrPhenotypicFeature", "Hep3B":"CellLine", "HepG2":"CellLine", "Huh7":"CellLine", "BMAL1":"GeneOrGeneProduct", "CLOCK":"GeneOrGeneProduct", "Bmal1":"GeneOrGeneProduct", "Clock":"GeneOrGeneProduct", "Wee1":"GeneOrGeneProduct", "p21":"GeneOrGeneProduct", "p21":"GeneOrGeneProduct", "HCC oncogenesis":"DiseaseOrPhenotypicFeature"},
{"cancer":"DiseaseOrPhenotypicFeature", "dubbed UV1":"GeneOrGeneProduct", "Yervoy": "ChemicalEntity", "pdivo":"ChemicalEntity", "malignant melanoma":"DiseaseOrPhenotypicFeature"},
{"temanogrel":"ChemicalEntity", " APD418":"GeneOrGeneProduct", "PF-06480605":"GeneOrGeneProduct", "RIST472":"GeneOrGeneProduct", "palmoplantar pustulosis":"DiseaseOrPhenotypicFeature", "hidradenitis suppurativa":"DiseaseOrPhenotypicFeature"},
{"proteins":"GeneOrGeneProduct", "peptides ":"GeneOrGeneProduct", "polypeptides":"GeneOrGeneProduct"},
{"COVID-19":"DiseaseOrPhenotypicFeature"},
{"lymphocytes ":"CellLine", "Natural Killer cells":"OrganismTaxon", "19q13.4":"GeneOrGeneProduct", "GNLY":"GeneOrGeneProduct", "MC1R":"GeneOrGeneProduct", "UVSSA":"GeneOrGeneProduct"},
{"PI3K":"GeneOrGeneProduct", " blood cancer":"DiseaseOrPhenotypicFeature", "parsaclisib":"ChemicalEntity", "JAK ":"GeneOrGeneProduct"},
{"diabetic macular edema":"DiseaseOrPhenotypicFeature", "DME":"DiseaseOrPhenotypicFeature", "UBX1325":"GeneOrGeneProduct", "anti-VEGF":"ChemicalEntity", "VEGF":"GeneOrGeneProduct"},
{"Alzheimer’s disease":"DiseaseOrPhenotypicFeature", "Parkinson’s diseases":"DiseaseOrPhenotypicFeature", "apolipoprotein E":"GeneOrGeneProduct", "ApoE":"GeneOrGeneProduct"},
{"insulin":"GeneOrGeneProduct", "Type 1 diabetes":"DiseaseOrPhenotypicFeature"},
{"Inorganic phosphate":"ChemicalEntity", "Pi":"ChemicalEntity", "AP-1":"GeneOrGeneProduct", "Cka":"GeneOrGeneProduct", "STRIPAK":"GeneOrGeneProduct"},
{"Gram-negative bacteria":"OrganismTaxon", "β-barrel proteins":"GeneOrGeneProduct", "OMP":"GeneOrGeneProduct"},
{"Astrocytes ":"CellLine", "neurons":"CellLine", "SAPAP3":"GeneOrGeneProduct", "OCD":"DiseaseOrPhenotypicFeature","cytoskeleton": "CellLine"},
{"DNMT3B":"GeneOrGeneProduct", "STAT3":"GeneOrGeneProduct", "osteoarthritis":"DiseaseOrPhenotypicFeature"},
{"hepatitis C":"DiseaseOrPhenotypicFeature", "CHC": "DiseaseOrPhenotypicFeature", "hepatic fibrosis":"DiseaseOrPhenotypicFeature", "cirrhosis":"DiseaseOrPhenotypicFeature", "hepatocellular carcinoma":"DiseaseOrPhenotypicFeature", "HCC": "DiseaseOrPhenotypicFeature","HCV":"DiseaseOrPhenotypicFeature"},
{"Gestational diabetes insipidus":"DiseaseOrPhenotypicFeature", "GDI": "DiseaseOrPhenotypicFeature", "polyuria":"DiseaseOrPhenotypicFeature", "polydypsia":"DiseaseOrPhenotypicFeature", "vasopressinase":"GeneOrGeneProduct", "arginine-vasopressin":"GeneOrGeneProduct", "AVP":"GeneOrGeneProduct",  "1-deamino-8-D arginine vasopressin":"ChemicalEntity",  "dDAVP":"ChemicalEntity", "preeclampsia":"DiseaseOrPhenotypicFeature", "gestational diabetes insipidus":"DiseaseOrPhenotypicFeature"},
{"T cells":"CellLine", "tumor":"DiseaseOrPhenotypicFeature", "pancreatic cancer":"DiseaseOrPhenotypicFeature", "PD-1":"GeneOrGeneProduct", "CTLA-4":"GeneOrGeneProduct", "TIM3":"GeneOrGeneProduct", "41BB":"GeneOrGeneProduct", "LAG":"GeneOrGeneProduct", "PD-1":"GeneOrGeneProduct", "LAG3":"GeneOrGeneProduct"}]
'''

'labelsnew = [{"Heat shock protein 47":"GeneOrGeneProduct", "HSP47":"GeneOrGeneProduct", "platelets":"CellLine", "collagen":"GeneOrGeneProduct", "blood clots":"DiseaseOrPhenotypicFeature"},\n{"Creutzfeldt-Jakob disease":"DiseaseOrPhenotypicFeature", "Parkinson\'s":"DiseaseOrPhenotypicFeature", "alpha-synuclein":"GeneOrGeneProduct", "Lewy bodies":"DiseaseOrPhenotypicFeature", "neurons":"CellLine", "dopamine":"ChemicalEntity", "tremors":"DiseaseOrPhenotypicFeature", "Alzheimer’s":"DiseaseOrPhenotypicFeature", "LRRK2":"GeneOrGeneProduct"},\n{"Polycomb protein complexes":"GeneOrGeneProduct", "PRC1":"GeneOrGeneProduct", "PRC2":"GeneOrGeneProduct", "neuron cell":"CellLine", "PRC2.1":"SequenceVariant", "PRC2.2":"SequenceVariant"},\n{"neuronal":"CellLine", "pentraxin protein":"GeneOrGeneProduct", "Nptx2":"GeneOrGeneProduct", "excitatory neurons":"CellLine", "Alzheimer’s disease":"DiseaseOrPhenotypicFeature", "frontotemporal dementia":"DiseaseOrPhenotypicFeature", "schizophrenia":"DiseaseOrPhen

In [254]:
labelsnew = [{"Heat shock protein 47":"GeneOrGeneProduct", "HSP47":"GeneOrGeneProduct", "platelets":"CellLine", "collagen":"GeneOrGeneProduct", "blood clots":"DiseaseOrPhenotypicFeature"},
{"Creutzfeldt-Jakob disease":"DiseaseOrPhenotypicFeature", "Parkinsons":"DiseaseOrPhenotypicFeature", "alpha-synuclein":"GeneOrGeneProduct", "Lewy bodies":"DiseaseOrPhenotypicFeature", "neurons":"CellLine", "dopamine":"ChemicalEntity", "tremors":"DiseaseOrPhenotypicFeature", "Alzheimers":"DiseaseOrPhenotypicFeature", "LRRK2":"GeneOrGeneProduct"},
{"Polycomb protein complexes":"GeneOrGeneProduct", "PRC1":"GeneOrGeneProduct", "PRC2":"GeneOrGeneProduct", "neuron cell":"CellLine", "PRC2.1":"SequenceVariant", "PRC2.2":"SequenceVariant"},
{"neuronal":"CellLine", "pentraxin protein":"GeneOrGeneProduct", "Nptx2":"GeneOrGeneProduct", "excitatory neurons":"CellLine", "Alzheimers disease":"DiseaseOrPhenotypicFeature", "frontotemporal dementia":"DiseaseOrPhenotypicFeature", "schizophrenia":"DiseaseOrPhenotypicFeature", "Down syndrome":"DiseaseOrPhenotypicFeature"},
{"cancer":"DiseaseOrPhenotypicFeature", "diabetes":"DiseaseOrPhenotypicFeature", "Type 2 diabetes":"DiseaseOrPhenotypicFeature"},
{"Venous thromboembolism":"DiseaseOrPhenotypicFeature", "VTE":"DiseaseOrPhenotypicFeature", "deep venous thrombosis":"DiseaseOrPhenotypicFeature", "pulmonary embolism":"DiseaseOrPhenotypicFeature", "spinal cord injury":"DiseaseOrPhenotypicFeature", "SCI":"DiseaseOrPhenotypicFeature", "heat shock protein 47":"GeneOrGeneProduct", "HSP47":"GeneOrGeneProduct", "platelets":"CellLine","neutrophil":"CellLine"},
{"Parkinsons disease":"DiseaseOrPhenotypicFeature", "dopaminergic deficit":"DiseaseOrPhenotypicFeature", "prodromal":"DiseaseOrPhenotypicFeature", "non-manifesting carriers":"DiseaseOrPhenotypicFeature", "α-synuclein":"GeneOrGeneProduct", "LRRK2":"GeneOrGeneProduct", "RBD":"DiseaseOrPhenotypicFeature", "hyposmia":"DiseaseOrPhenotypicFeature"},
{"Polycomb repressive complex 2":"GeneOrGeneProduct", "EED":"GeneOrGeneProduct", "SUZ12":"GeneOrGeneProduct", "EZH1/2":"GeneOrGeneProduct", "histone H3":"GeneOrGeneProduct", "PRC2.1":"SequenceVariant", "PCL1–3":"GeneOrGeneProduct", "EPOP":"GeneOrGeneProduct", "PALI1/2":"GeneOrGeneProduct", "PRC2.2":"SequenceVariant", "AEBP2":"GeneOrGeneProduct", "JARID2":"GeneOrGeneProduct", "mouse embryonic stem cells":"CellLine", "Polycomb":"GeneOrGeneProduct", "RING1A/B":"GeneOrGeneProduct", "PCLs":"GeneOrGeneProduct"},
{"microglial":"CellLine", "Alzheimers disease":"DiseaseOrPhenotypicFeature", "AD":"DiseaseOrPhenotypicFeature", "frontotemporal dementia":"DiseaseOrPhenotypicFeature", "FTD":"DiseaseOrPhenotypicFeature", "pentraxin":"GeneOrGeneProduct", "Nptx2":"GeneOrGeneProduct", "C1q":"GeneOrGeneProduct", "Nptx2-C1q":"GeneOrGeneProduct", "C3":"GeneOrGeneProduct"},
{"Type 2 diabetes mellitus":"DiseaseOrPhenotypicFeature", "T2D":"DiseaseOrPhenotypicFeature"},
{"pluripotent stem cells":"CellLine", "endothelial cells":"CellLine", "mesodermal cells":"CellLine", "inflammation":"DiseaseOrPhenotypicFeature", "tumor formation":"DiseaseOrPhenotypicFeature", "Down syndrome":"DiseaseOrPhenotypicFeature", "myosin":"GeneOrGeneProduct"},
{"R805":"ChemicalEntity", "CX-011":"ChemicalEntity", "inflammation":"DiseaseOrPhenotypicFeature", "osteoarthritis":"DiseaseOrPhenotypicFeature", "Cartilage erosion":"DiseaseOrPhenotypicFeature", "gp130":"GeneOrGeneProduct"},
{"Down syndrome":"DiseaseOrPhenotypicFeature", "DS":"DiseaseOrPhenotypicFeature", "trisomy 21":"SequenceVariant", "T21":"SequenceVariant", "cognitive impairment":"DiseaseOrPhenotypicFeature", "muscle hypotonia":"DiseaseOrPhenotypicFeature", "heart defects":"DiseaseOrPhenotypicFeature", "solid tumor formation":"DiseaseOrPhenotypicFeature", "iPSC-derived stromal cells":"CellLine", "mesodermal progenitors":"CellLine", "stromal progenitor":"CellLine", "actin":"GeneOrGeneProduct", "myosin":"GeneOrGeneProduct", "Collagens":"GeneOrGeneProduct", "Galectin-1":"GeneOrGeneProduct", "Fibronectin":"GeneOrGeneProduct", "Heparan":"GeneOrGeneProduct", "Sulfate":"ChemicalEntity", "LOX":"GeneOrGeneProduct", "FAK1":"GeneOrGeneProduct", "USP16":"GeneOrGeneProduct", "S1P complexes":"GeneOrGeneProduct", "miR-24-4":"GeneOrGeneProduct", "miR-21":"GeneOrGeneProduct"},
{"interleukin 6":"GeneOrGeneProduct", "IL-6":"GeneOrGeneProduct", "cytokines":"GeneOrGeneProduct", "glycoprotein 130":"GeneOrGeneProduct", "gp130":"GeneOrGeneProduct", "signaling tyrosine 814":"GeneOrGeneProduct", "Y814":"GeneOrGeneProduct", "F814":"GeneOrGeneProduct", "SRC":"GeneOrGeneProduct", "MAPK":"GeneOrGeneProduct", "R805":"ChemicalEntity", "hair follicle neogenesis":"DiseaseOrPhenotypicFeature", "dermal regeneration":"DiseaseOrPhenotypicFeature", "osteoarthritis":"DiseaseOrPhenotypicFeature"},
{"human adenovirus 52-kDa protein":"GeneOrGeneProduct", "capsid":"GeneOrGeneProduct", "adenovirus":"OrganismTaxon"},
{"HDAC":"GeneOrGeneProduct", "LKB1":"GeneOrGeneProduct", "cancer":"DiseaseOrPhenotypicFeature", "tumor":"DiseaseOrPhenotypicFeature", "HDAC3":"GeneOrGeneProduct","NSCLC":"DiseaseOrPhenotypicFeature"},
{"USB1":"GeneOrGeneProduct", "hematopoietic failure":"DiseaseOrPhenotypicFeature", "poikiloderma":"DiseaseOrPhenotypicFeature", "neutropenia":"DiseaseOrPhenotypicFeature", "PN": "DiseaseOrPhenotypicFeature", "U6 small nuclear RNA":"GeneOrGeneProduct", "pre-mRNA":"GeneOrGeneProduct", "c.531_delA":"SequenceVariant", "microRNA":"GeneOrGeneProduct", "miRNA": "GeneOrGeneProduct", "PAPD5/7":"GeneOrGeneProduct", "miRNA 3′-end adenylation":"SequenceVariant", "hematopoiesis":"DiseaseOrPhenotypicFeature", "deadenylase":"GeneOrGeneProduct"},
{"microRNA":"GeneOrGeneProduct", "PAPD5":"GeneOrGeneProduct", "RG7834":"ChemicalEntity", "RO7020322":"ChemicalEntity", "hepatitis B":"DiseaseOrPhenotypicFeature", "UB1":"SequenceVariant", "RG7834":"ChemicalEntity", "microRNA":"GeneOrGeneProduct", "neutrophil":"CellLine", "PN":"DiseaseOrPhenotypicFeature"},
{"CRISPR-Cas13 proteins":"GeneOrGeneProduct", "RNA-guided RNA nucleases":"GeneOrGeneProduct", "phage":"OrganismTaxon", "RNA":"GeneOrGeneProduct", "LbuCas13a":"GeneOrGeneProduct", "Cas13a":"GeneOrGeneProduct"},
{"microglia":"CellLine", "Huntingtons disease":"DiseaseOrPhenotypicFeature", "tau":"GeneOrGeneProduct", "dementia":"DiseaseOrPhenotypicFeature"},
{"insomnia":"DiseaseOrPhenotypicFeature", "restless leg syndrome":"DiseaseOrPhenotypicFeature", "FABP7":"GeneOrGeneProduct", "ZFYVE28":"GeneOrGeneProduct"},
{"Uveitis":"DiseaseOrPhenotypicFeature", "glaucoma":"DiseaseOrPhenotypicFeature", "cancer":"DiseaseOrPhenotypicFeature", "P-TEFb ":"GeneOrGeneProduct", " T helper cells":"CellLine"},
{"otulipenia":"DiseaseOrPhenotypicFeature", "OTULIN":"GeneOrGeneProduct", "ubiquitin":"GeneOrGeneProduct", "rheumatoid arthritis":"DiseaseOrPhenotypicFeature"},
{"Joubert syndrome":"DiseaseOrPhenotypicFeature", "CEP290":"GeneOrGeneProduct", "kidney cells":"CellLine", "kidney disease":"DiseaseOrPhenotypicFeature"},
{"Parkinsons disease":"DiseaseOrPhenotypicFeature",  "PD":"DiseaseOrPhenotypicFeature", "c-Abl ":"GeneOrGeneProduct", "leukemia":"DiseaseOrPhenotypicFeature", "α-synuclein":"GeneOrGeneProduct"},
{"tauopathies":"DiseaseOrPhenotypicFeature", "Huntingtons disease":"DiseaseOrPhenotypicFeature", "HD":"DiseaseOrPhenotypicFeature", "aggregate-prone proteins":"GeneOrGeneProduct", "neurodegeneration":"DiseaseOrPhenotypicFeature", "Alzheimers disease":"DiseaseOrPhenotypicFeature", "microglia":"CellLine", "cytokines":"GeneOrGeneProduct", "chemokines":"GeneOrGeneProduct", "neuronal receptors":"GeneOrGeneProduct", "CCL3":"GeneOrGeneProduct", "CCL4":"GeneOrGeneProduct", "CCL5":"GeneOrGeneProduct", "CCR5":"GeneOrGeneProduct"},
{"insomnia":"DiseaseOrPhenotypicFeature", "restless legs syndrome":"DiseaseOrPhenotypicFeature", "coronary artery disease":"DiseaseOrPhenotypicFeature", },
{"cancer":"DiseaseOrPhenotypicFeature", "chimeric antigen receptor":"GeneOrGeneProduct", "T cells": "CellLine", "refractory B cell leukemia":"DiseaseOrPhenotypicFeature", "CAR Tregs":"GeneOrGeneProduct", "NK cells":"GeneOrGeneProduct"},
{"Otulipenia":"DiseaseOrPhenotypicFeature", "Inflammation":"DiseaseOrPhenotypicFeature", "bacteria":"OrganismTaxon", "viruses":"OrganismTaxon", "OTULIN":"GeneOrGeneProduct", "ubiquitin":"GeneOrGeneProduct", "cytokines":"GeneOrGeneProduct"},
{"renal ciliopathies":"DiseaseOrPhenotypicFeature", "cystic kidney disease":"DiseaseOrPhenotypicFeature", "CEP290":"GeneOrGeneProduct", "end stage renal failure":"DiseaseOrPhenotypicFeature", "ESRF":"DiseaseOrPhenotypicFeature", "CEP290-associated JBTS":"DiseaseOrPhenotypicFeature", "antisense oligonucleotide":"GeneOrGeneProduct", "ASO": "GeneOrGeneProduct", "41, G1890":"SequenceVariant"},
{"c-Abl ":"GeneOrGeneProduct", "Parkinsons disease":"DiseaseOrPhenotypicFeature", "leukemia":"DiseaseOrPhenotypicFeature"},
{"METTL3":"GeneOrGeneProduct", "RNA N6-methyladenosine":"SequenceVariant", "m6A, methyltransferase":"GeneOrGeneProduct", "m6A methylation":"SequenceVariant", "mRNA":"GeneOrGeneProduct", "Alzheimers":"DiseaseOrPhenotypicFeature", "myeloid cells":"CellLine", "macrophages":"OrganismTaxon", "microglia":"CellLine"},
{"Hepatocellular carcinoma":"DiseaseOrPhenotypicFeature", "hepatocarcinogenesis":"DiseaseOrPhenotypicFeature", "Hep3B":"CellLine", "HepG2":"CellLine", "Huh7":"CellLine", "BMAL1":"GeneOrGeneProduct", "CLOCK":"GeneOrGeneProduct", "Bmal1":"GeneOrGeneProduct", "Clock":"GeneOrGeneProduct", "Wee1":"GeneOrGeneProduct", "p21":"GeneOrGeneProduct", "p21":"GeneOrGeneProduct", "HCC oncogenesis":"DiseaseOrPhenotypicFeature"},
{"cancer":"DiseaseOrPhenotypicFeature", "dubbed UV1":"GeneOrGeneProduct", "Yervoy": "ChemicalEntity", "pdivo":"ChemicalEntity", "malignant melanoma":"DiseaseOrPhenotypicFeature"},
{"temanogrel":"ChemicalEntity", " APD418":"GeneOrGeneProduct", "PF-06480605":"GeneOrGeneProduct", "RIST472":"GeneOrGeneProduct", "palmoplantar pustulosis":"DiseaseOrPhenotypicFeature", "hidradenitis suppurativa":"DiseaseOrPhenotypicFeature"},
{"proteins":"GeneOrGeneProduct", "peptides ":"GeneOrGeneProduct", "polypeptides":"GeneOrGeneProduct"},
{"COVID-19":"DiseaseOrPhenotypicFeature"},
{"lymphocytes ":"CellLine", "Natural Killer cells":"OrganismTaxon", "19q13.4":"GeneOrGeneProduct", "GNLY":"GeneOrGeneProduct", "MC1R":"GeneOrGeneProduct", "UVSSA":"GeneOrGeneProduct"},
{"PI3K":"GeneOrGeneProduct", " blood cancer":"DiseaseOrPhenotypicFeature", "parsaclisib":"ChemicalEntity", "JAK ":"GeneOrGeneProduct"},
{"diabetic macular edema":"DiseaseOrPhenotypicFeature", "DME":"DiseaseOrPhenotypicFeature", "UBX1325":"GeneOrGeneProduct", "anti-VEGF":"ChemicalEntity", "VEGF":"GeneOrGeneProduct"},
{"Alzheimers disease":"DiseaseOrPhenotypicFeature", "Parkinsons diseases":"DiseaseOrPhenotypicFeature", "apolipoprotein E":"GeneOrGeneProduct", "ApoE":"GeneOrGeneProduct"},
{"insulin":"GeneOrGeneProduct", "Type 1 diabetes":"DiseaseOrPhenotypicFeature"},
{"Inorganic phosphate":"ChemicalEntity", "Pi":"ChemicalEntity", "AP-1":"GeneOrGeneProduct", "Cka":"GeneOrGeneProduct", "STRIPAK":"GeneOrGeneProduct"},
{"Gram-negative bacteria":"OrganismTaxon", "β-barrel proteins":"GeneOrGeneProduct", "OMP":"GeneOrGeneProduct"},
{"Astrocytes ":"CellLine", "neurons":"CellLine", "SAPAP3":"GeneOrGeneProduct", "OCD":"DiseaseOrPhenotypicFeature","cytoskeleton": "CellLine"},
{"DNMT3B":"GeneOrGeneProduct", "STAT3":"GeneOrGeneProduct", "osteoarthritis":"DiseaseOrPhenotypicFeature"},
{"hepatitis C":"DiseaseOrPhenotypicFeature", "CHC": "DiseaseOrPhenotypicFeature", "hepatic fibrosis":"DiseaseOrPhenotypicFeature", "cirrhosis":"DiseaseOrPhenotypicFeature", "hepatocellular carcinoma":"DiseaseOrPhenotypicFeature", "HCC": "DiseaseOrPhenotypicFeature","HCV":"DiseaseOrPhenotypicFeature"},
{"Gestational diabetes insipidus":"DiseaseOrPhenotypicFeature", "GDI": "DiseaseOrPhenotypicFeature", "polyuria":"DiseaseOrPhenotypicFeature", "polydypsia":"DiseaseOrPhenotypicFeature", "vasopressinase":"GeneOrGeneProduct", "arginine-vasopressin":"GeneOrGeneProduct", "AVP":"GeneOrGeneProduct",  "1-deamino-8-D arginine vasopressin":"ChemicalEntity",  "dDAVP":"ChemicalEntity", "preeclampsia":"DiseaseOrPhenotypicFeature", "gestational diabetes insipidus":"DiseaseOrPhenotypicFeature"},
{"T cells":"CellLine", "tumor":"DiseaseOrPhenotypicFeature", "pancreatic cancer":"DiseaseOrPhenotypicFeature", "PD-1":"GeneOrGeneProduct", "CTLA-4":"GeneOrGeneProduct", "TIM3":"GeneOrGeneProduct", "41BB":"GeneOrGeneProduct", "LAG":"GeneOrGeneProduct", "PD-1":"GeneOrGeneProduct", "LAG3":"GeneOrGeneProduct"}
]




In [255]:
data['Labels'] = labelsnew

In [256]:
entitieslist = []
for d in data['Labels']:
  entities = d.keys()
  entitieslist.append(entities)
data['Entity'] = entitieslist

In [257]:
labelslist = []
for i in data['Labels']:
  labels = i.values()
  labelslist.append(labels)
data['Label'] = labelslist

In [258]:
# redux
offsets_list = []
fix_this_list = []
for i, row in enumerate(data.iterrows()):
  offsets = []
  for entity in row[1]['Entity']:
    pattern = re.compile(entity)
    S = row[1]['text/abstract']
    r = pattern.search(S)
    if r is not None:
      offsets.append("[{0}, {1}]".format(r.start(), r.end() - 1))
    else:
      fix_this_list.append((i, pattern))
  offsets_list.append(offsets)
data['Offsets'] = offsets_list

In [259]:
print(fix_this_list)

[]


In [260]:
data.head()

,Type,Website,text/abstract,Labels,Entity,Label,Offsets
0,article,https://www.sciencenews.org/article/hibernatin...,"Heat shock protein 47, or HSP47, is normally f...","{'Heat shock protein 47': 'GeneOrGeneProduct',...","(Heat shock protein 47, HSP47, platelets, coll...","(GeneOrGeneProduct, GeneOrGeneProduct, CellLin...","[[0, 20], [26, 30], [139, 147], [174, 181], [7..."
1,article,https://www.fiercebiotech.com/research/new-tes...,"SYNTap is a seed amplification assay, or SAA, ...",{'Creutzfeldt-Jakob disease': 'DiseaseOrPhenot...,"(Creutzfeldt-Jakob disease, Parkinsons, alpha-...","(DiseaseOrPhenotypicFeature, DiseaseOrPhenotyp...","[[123, 147], [255, 264], [277, 291], [347, 357..."
2,article,https://www.fiercebiotech.com/research/discove...,Professor Adrian Bracken and his team had been...,{'Polycomb protein complexes': 'GeneOrGeneProd...,"(Polycomb protein complexes, PRC1, PRC2, neuro...","(GeneOrGeneProduct, GeneOrGeneProduct, GeneOrG...","[[81, 106], [122, 125], [131, 134], [251, 261]..."
3,article,https://www.fiercebiotech.com/research/returni...,The breakthrough from researchers including at...,"{'neuronal': 'CellLine', 'pentraxin protein': ...","(neuronal, pentraxin protein, Nptx2, excitator...","(CellLine, GeneOrGeneProduct, GeneOrGeneProduc...","[[154, 161], [163, 179], [181, 185], [441, 458..."
4,article,https://www.fiercebiotech.com/medtech/new-test...,"Currently being explored in cancer testing, DN...","{'cancer': 'DiseaseOrPhenotypicFeature', 'diab...","(cancer, diabetes, Type 2 diabetes)","(DiseaseOrPhenotypicFeature, DiseaseOrPhenotyp...","[[28, 33], [526, 533], [738, 752]]"


In [261]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset

import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def tokenize_text(txt):
  if '-' in txt:
     txt = txt.replace("-", " - ")
  if '/' in txt:
     txt = txt.replace("/", " / ")
  if '+' in txt:
     txt = txt.replace("+", " + ")
  if '.' in txt:
     txt = txt.replace(".", " . ")
  return word_tokenize(txt)

def tokenize_passage(p):
  p["tokens"]=p["text/abstract"]
  for i in range(len(p["tokens"])):
    p["tokens"][i]=tokenize_text(p["tokens"][i])
  return(p)

def corrLabels(d, label2id):
  ner_full_list = []
  for i in range(len(d)):
    ner_tags=[]
    if eval(data["Offsets"][i][0])[0] != 0:
        begin_ent = data['text/abstract'][i][0: eval(data["Offsets"][i][0])[0]]
        token_begin_ent = tokenize_text(begin_ent)
        for k in range(len(token_begin_ent)):
          ner_tags.append(0)
    for j in range(len(list(data['Entity'][i]))):
      ent = data["text/abstract"][i][eval(data["Offsets"][i][j])[0]: eval(data["Offsets"][i][j])[1]+1]
      token_ent = tokenize_text(ent)
      ent_label = data['Labels'][i][ent]
      ent_tag = label2id[ent_label]
      for k in range(len(token_ent)):
        ner_tags.append(ent_tag)
      try:
        between_ent = data["text/abstract"][i][eval(data["Offsets"][i][j])[1]+1: eval(data["Offsets"][i][j+1])[0]]
        token_between_ent = tokenize_text(between_ent)
        for k in range(len(token_between_ent)):
          ner_tags.append(0)
      except Exception:
        pass
    #print(eval(data["Offsets"][i][j])[1]+1)
    if eval(data["Offsets"][i][j])[1]+1 != len(data['text/abstract']):
      remainder_text = data['text/abstract'][i][eval(data["Offsets"][i][j])[1]+1:]
      token_remainder_text = tokenize_text(remainder_text)
      for k in range(len(token_remainder_text)):
        ner_tags.append(0)
    ner_full_list.append(ner_tags)
  d['ner_tags'] = ner_full_list
  return d

def remove_excess(df, id2label):
  df['ner_tags_labels'] = df['ner_tags'].apply(lambda x: [id2label[int(xx)] for xx in x])
  idx_to_drop = []
  for idx in range(len(df)):
      tt = df[['tokens',  'ner_tags_labels']].iloc[idx].to_dict()
      a, b = len(tt['tokens']), len(tt['ner_tags_labels'])
      if a != b:
          df['ner_tags_labels'][idx] = df['ner_tags_labels'][idx][0: len(tt['tokens'])]
  return df

def drop_misaligned(df, id2label):
  df['ner_tags_labels'] = df['ner_tags'].apply(lambda x: [id2label[int(xx)] for xx in x])
  idx_to_drop = []
  for idx in range(len(df)):
      tt = df[['tokens',  'ner_tags_labels']].iloc[idx].to_dict()
      a, b = len(tt['tokens']), len(tt['ner_tags_labels'])
      if a != b:
          idx_to_drop.append(idx)
  print(idx_to_drop)


def tokenize_and_align_labels(tokenizer, examples):
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs
  
def ner_tags_creator(df, id2label):
  df['ner_tags_labels'] = df['ner_tags'].apply(lambda x: [id2label[int(xx)] for xx in x])
  return df

def ner_tags_updater(df, label2id):
  df['ner_tags'] = df['ner_tags_labels'].apply(lambda x: [label2id[xx] for xx in x])
  return df

def BI0_updater(data):
    ner_tags_list = list(data['ner_tags_labels'])
    ner_tags_list2 = []
    ner_tags_new = []
    for passage in ner_tags_list:
      passage2 = []
      for index2, word in enumerate(passage):
        # first one
        if index2 == 0 and word != 'None':
          passage2.append('BBBB' + str(word))
          #print('B-' + str(word))
        elif index2 == 0 and word == 'None':
          passage2.append(word)
         # print(word)
        # the rest
        else:
          if word == passage[index2-1] and word != 'None':
            passage2.append('IIII' + str(word))
            #print('I-' + str(word))
          elif word != passage[index2-1] and word != 'None':
            passage2.append('BBBB' + str(word))
            #print('B-' + str(word))
          elif word == 'None':
            passage2.append(word)
            #print(word)
      ner_tags_list2.append(passage2)
    ner_tags_series = pd.Series(ner_tags_list2) 
    data['ner_tags_labels'] = ner_tags_series
    
    return data

dataset = load_dataset("bigbio/biored")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/3 [00:00<?, ?it/s]

In [262]:
# run only once
label2id={"None":0,"GeneOrGeneProduct":1,"DiseaseOrPhenotypicFeature":2,"ChemicalEntity":3,"OrganismTaxon":4,"SequenceVariant":5,"CellLine":6}
id2label={0:"None",1:"GeneOrGeneProduct",2:"DiseaseOrPhenotypicFeature",3:"ChemicalEntity",4:"OrganismTaxon",5:"SequenceVariant",6:"CellLine"}

testing_set = tokenize_passage(data)
testing_set = corrLabels(testing_set, label2id)
testing_set = ner_tags_creator(testing_set, id2label)

In [263]:
# run only once
id2label={0:"None",1:"BBBBGeneOrGeneProduct",2:"IIIIGeneOrGeneProduct",3:"BBBBDiseaseOrPhenotypicFeature",4: "IIIIDiseaseOrPhenotypicFeature",5:"BBBBChemicalEntity",6:"IIIIChemicalEntity", 7: "BBBBOrganismTaxon", 8:"IIIIOrganismTaxon", 9:"BBBBSequenceVariant", 10:"IIIISequenceVariant", 11: "BBBBCellLine", 12:"IIIICellLine"  }
label2id={"None":0,"BBBBGeneOrGeneProduct":1,"IIIIGeneOrGeneProduct":2, "BBBBDiseaseOrPhenotypicFeature":3, "IIIIDiseaseOrPhenotypicFeature":4, "BBBBChemicalEntity":5,"IIIIChemicalEntity":6, "BBBBOrganismTaxon":7,"IIIIOrganismTaxon":8, "BBBBSequenceVariant":9,"IIIISequenceVariant":10, "BBBBCellLine":11,"IIIICellLine":12}
testing_set = BI0_updater(testing_set)
testing_set = ner_tags_updater(testing_set, label2id)
testing_set = remove_excess(testing_set, id2label)
#testing_set = drop_misaligned(testing_set, id2label)

In [264]:
testing_set.head()

,Type,Website,text/abstract,Labels,Entity,Label,Offsets,tokens,ner_tags,ner_tags_labels
0,article,https://www.sciencenews.org/article/hibernatin...,"Heat shock protein 47, or HSP47, is normally f...","{'Heat shock protein 47': 'GeneOrGeneProduct',...","(Heat shock protein 47, HSP47, platelets, coll...","(GeneOrGeneProduct, GeneOrGeneProduct, CellLin...","[[0, 20], [26, 30], [139, 147], [174, 181], [7...","[Heat, shock, protein, 47, ,, or, HSP47, ,, is...","[1, 2, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[BBBBGeneOrGeneProduct, IIIIGeneOrGeneProduct,..."
1,article,https://www.fiercebiotech.com/research/new-tes...,"SYNTap is a seed amplification assay, or SAA, ...",{'Creutzfeldt-Jakob disease': 'DiseaseOrPhenot...,"(Creutzfeldt-Jakob disease, Parkinsons, alpha-...","(DiseaseOrPhenotypicFeature, DiseaseOrPhenotyp...","[[123, 147], [255, 264], [277, 291], [347, 357...","[SYNTap, is, a, seed, amplification, assay, ,,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[None, None, None, None, None, None, None, Non..."
2,article,https://www.fiercebiotech.com/research/discove...,Professor Adrian Bracken and his team had been...,{'Polycomb protein complexes': 'GeneOrGeneProd...,"(Polycomb protein complexes, PRC1, PRC2, neuro...","(GeneOrGeneProduct, GeneOrGeneProduct, GeneOrG...","[[81, 106], [122, 125], [131, 134], [251, 261]...","[Professor, Adrian, Bracken, and, his, team, h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[None, None, None, None, None, None, None, Non..."
3,article,https://www.fiercebiotech.com/research/returni...,The breakthrough from researchers including at...,"{'neuronal': 'CellLine', 'pentraxin protein': ...","(neuronal, pentraxin protein, Nptx2, excitator...","(CellLine, GeneOrGeneProduct, GeneOrGeneProduc...","[[154, 161], [163, 179], [181, 185], [441, 458...","[The, breakthrough, from, researchers, includi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[None, None, None, None, None, None, None, Non..."
4,article,https://www.fiercebiotech.com/medtech/new-test...,"Currently being explored in cancer testing, DN...","{'cancer': 'DiseaseOrPhenotypicFeature', 'diab...","(cancer, diabetes, Type 2 diabetes)","(DiseaseOrPhenotypicFeature, DiseaseOrPhenotyp...","[[28, 33], [526, 533], [738, 752]]","[Currently, being, explored, in, cancer, testi...","[0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[None, None, None, None, BBBBDiseaseOrPhenotyp..."


In [265]:
for i in range(len(data)):
  print(len(testing_set['Offsets'].loc[i]), len(testing_set['Label'].loc[i]))

5 5
9 9
6 6
8 8
3 3
10 10
8 8
16 16
10 10
2 2
7 7
6 6
24 24
14 14
3 3
6 6
14 14
8 8
6 6
4 4
4 4
5 5
4 4
4 4
5 5
14 14
3 3
6 6
7 7
9 9
3 3
9 9
12 12
5 5
6 6
3 3
1 1
6 6
4 4
5 5
4 4
2 2
5 5
3 3
5 5
3 3
7 7
11 11
9 9


In [266]:
len(testing_set)

49

In [267]:
len(testing_set['ner_tags_labels'][24])

332

In [268]:
len(testing_set['tokens'][24])

332

In [269]:
for k in range(len(testing_set['ner_tags_labels'][5])):
  try:
    print((testing_set['ner_tags_labels'][5][k], testing_set['tokens'][5][k]))
  except Exception:
    print(testing_set['ner_tags_labels'][5][k])

('BBBBDiseaseOrPhenotypicFeature', 'Venous')
('IIIIDiseaseOrPhenotypicFeature', 'thromboembolism')
('None', '(')
('BBBBDiseaseOrPhenotypicFeature', 'VTE')
('None', ')')
('None', 'comprising')
('BBBBDiseaseOrPhenotypicFeature', 'deep')
('IIIIDiseaseOrPhenotypicFeature', 'venous')
('IIIIDiseaseOrPhenotypicFeature', 'thrombosis')
('None', 'and')
('BBBBDiseaseOrPhenotypicFeature', 'pulmonary')
('IIIIDiseaseOrPhenotypicFeature', 'embolism')
('None', 'is')
('None', 'a')
('None', 'major')
('None', 'cause')
('None', 'of')
('None', 'morbidity')
('None', 'and')
('None', 'mortality')
('None', '.')
('None', 'Short')
('None', '-')
('None', 'term')
('None', 'immobility')
('None', '-')
('None', 'related')
('None', 'conditions')
('None', 'are')
('None', 'a')
('None', 'major')
('None', 'risk')
('None', 'factor')
('None', 'for')
('None', 'the')
('None', 'development')
('None', 'of')
('None', 'VTE')
('None', '.')
('None', 'Paradoxically')
('None', ',')
('None', 'long')
('None', '-')
('None', 'term')
('No

In [270]:
testing_set['text/abstract'][13]

'Adult mammals are incapable of multitissue regeneration, and augmentation of this potential may shift current therapeutic paradigms. We found that a common co-receptor of interleukin 6 (IL-6) cytokines, glycoprotein 130 (gp130), serves as a major nexus integrating various context-specific signaling inputs to either promote regenerative outcomes or aggravate disease progression. Via genetic and pharmacological experiments in vitro and in vivo, we demonstrated that a signaling tyrosine 814 (Y814) within gp130 serves as a major cellular stress sensor. Mice with constitutively inactivated Y814 (F814) were resistant to surgically induced osteoarthritis as reflected by reduced loss of proteoglycans, reduced synovitis, and synovial fibrosis. The F814 mice also exhibited enhanced regenerative, not reparative, responses after wounding in the skin. In addition, pharmacological modulation of gp130 Y814 upstream of the SRC and MAPK circuit by a small molecule, R805, elicited a protective effect o

In [271]:
for k in range(len(testing_set['ner_tags'][2])):
  try:
    print((testing_set['ner_tags'][2][k], testing_set['tokens'][2][k]))
  except Exception:
    print(testing_set['ner_tags'][2][k])

(0, 'Professor')
(0, 'Adrian')
(0, 'Bracken')
(0, 'and')
(0, 'his')
(0, 'team')
(0, 'had')
(0, 'been')
(0, 'perplexed')
(0, 'by')
(0, 'a')
(0, '“')
(0, 'puzzle')
(0, '”')
(0, 'involving')
(1, 'Polycomb')
(2, 'protein')
(2, 'complexes')
(0, ',')
(0, 'specifically')
(1, 'PRC1')
(0, 'and')
(1, 'PRC2')
(0, '.')
(0, 'These')
(0, 'proteins')
(0, 'are')
(0, '“')
(0, 'strict')
(0, 'librarians')
(0, '”')
(0, 'inside')
(0, 'cells')
(0, ',')
(0, 'blocking')
(0, 'access')
(0, 'to')
(0, 'certain')
(0, 'areas')
(0, 'of')
(0, 'the')
(0, 'genetic')
(0, 'library—so')
(0, 'a')
(11, 'neuron')
(12, 'cell')
(0, 'cant')
(0, 'access')
(0, 'a')
(0, 'muscle')
(0, 'gene')
(0, ',')
(0, 'confusing')
(0, 'its')
(0, 'cellular')
(0, 'identity')
(0, ',')
(0, 'for')
(0, 'instance')
(0, ',')
(0, 'Ph')
(0, '.')
(0, 'D')
(0, '.')
(0, 'student')
(0, 'Ellen')
(0, 'Tuck')
(0, ',')
(0, 'who')
(0, 'works')
(0, 'in')
(0, 'the')
(0, 'Bracken')
(0, 'lab')
(0, ',')
(0, 'said')
(0, 'in')
(0, 'a')
(0, 'press')
(0, 'release')
(0, '.

In [272]:
testing_set['Labels'][1]

{'Creutzfeldt-Jakob disease': 'DiseaseOrPhenotypicFeature',
 'Parkinsons': 'DiseaseOrPhenotypicFeature',
 'alpha-synuclein': 'GeneOrGeneProduct',
 'Lewy bodies': 'DiseaseOrPhenotypicFeature',
 'neurons': 'CellLine',
 'dopamine': 'ChemicalEntity',
 'tremors': 'DiseaseOrPhenotypicFeature',
 'Alzheimers': 'DiseaseOrPhenotypicFeature',
 'LRRK2': 'GeneOrGeneProduct'}

In [308]:
from transformers import pipeline
from datasets import load_dataset

id2label={0:"None",1:"BBBBGeneOrGeneProduct",2:"IIIIGeneOrGeneProduct",3:"BBBBDiseaseOrPhenotypicFeature",4: "IIIIDiseaseOrPhenotypicFeature",5:"BBBBChemicalEntity",6:"IIIIChemicalEntity", 7: "BBBBOrganismTaxon", 8:"IIIIOrganismTaxon", 9:"BBBBSequenceVariant", 10:"IIIISequenceVariant", 11: "BBBBCellLine", 12:"IIIICellLine"  }
label2id={"None":0,"BBBBGeneOrGeneProduct":1,"IIIIGeneOrGeneProduct":2, "BBBBDiseaseOrPhenotypicFeature":3, "IIIIDiseaseOrPhenotypicFeature":4, "BBBBChemicalEntity":5,"IIIIChemicalEntity":6, "BBBBOrganismTaxon":7,"IIIIOrganismTaxon":8, "BBBBSequenceVariant":9,"IIIISequenceVariant":10, "BBBBCellLine":11,"IIIICellLine":12}

classifier = pipeline("ner", model="J4YL19/biored")

full_ent_list = []
for txt in testing_set['text/abstract']:
  res = classifier(txt)
  ent_list = []
  for ent_dict in res:
    ent_list.append(ent_dict['entity'])
  full_ent_list.extend(ent_list)

#res = classifier(txt)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [309]:
for i in range(len(testing_set)):
  txt = testing_set['text/abstract'][i]
  res = classifier(txt)
  if len([x['entity'] for x in res]) != len(testing_set['ner_tags_labels']):
    print(len([x['entity'] for x in res]) - len(testing_set['ner_tags_labels']))

199
307
146
88
93
120
368
208
165
221
101
240
414
279
168
94
137
132
171
95
231
102
144
221
295
437
169
206
154
217
187
191
219
161
173
104
168
147
169
169
248
267
225
129
261
108
126
107
164


In [310]:
txt = testing_set['text/abstract'][0]
res = classifier(txt)
print("tokens:")
print(str([x['word'] for x in res]))
print("predictions:")
print(str([x['entity'] for x in res]))
print("labels:")
print(testing_set['ner_tags_labels'][0])

tokens:
['heat', 'shock', 'protein', '47', ',', 'or', 'hsp', '##47', ',', 'is', 'normally', 'found', 'in', 'the', 'cells', 'that', 'make', 'up', 'connective', 'tissues', 'like', 'bone', 'and', 'cartilage', '.', 'its', 'also', 'found', 'in', 'platelets', ',', 'where', 'hsp', '##47', 'attach', '##es', 'to', 'collagen', ',', 'a', 'protein', 'that', 'helps', 'platelets', 'stick', 'together', '.', 'this', 'is', 'helpful', 'when', 'the', 'body', 'responds', 'to', 'a', 'cut', 'or', 'other', 'injury', ';', 'its', 'dangerous', 'when', 'a', 'cl', '##ump', 'of', 'platelets', 'blocks', 'blood', 'flow', 'to', 'the', 'lungs', '.', 'potential', 'drugs', 'based', 'on', 'this', 'study', '##s', 'finding', 'would', 'aim', 'to', 'stop', 'hsp', '##47', 'from', 'interacting', 'with', 'proteins', 'or', 'immune', 'cells', 'that', 'spark', 'clot', '##s', ',', 'say', '##s', 'tob', '##ias', 'pet', '##zo', '##ld', ',', 'a', 'cardio', '##lo', '##gi', '##st', 'at', 'university', 'hospital', 'at', 'lu', '##d', '##wi

In [323]:
for i in range(len([x['entity'] for x in res])):
  try:
    print((testing_set['tokens'][0][i], [x['word'] for x in res][i]))
    print(([x['entity'] for x in res][i], testing_set['ner_tags_labels'][0][i]))
  except Exception:
    print([x['entity'] for x in res][i])

('Heat', 'heat')
('BBGeneOrGeneProduct', 'BBBBGeneOrGeneProduct')
('shock', 'shock')
('IIGeneOrGeneProduct', 'IIIIGeneOrGeneProduct')
('protein', 'protein')
('IIGeneOrGeneProduct', 'IIIIGeneOrGeneProduct')
('47', '47')
('IIGeneOrGeneProduct', 'IIIIGeneOrGeneProduct')
(',', ',')
('None', 'None')
('or', 'or')
('None', 'None')
('HSP47', 'hsp')
('BBGeneOrGeneProduct', 'BBBBGeneOrGeneProduct')
(',', '##47')
('IIGeneOrGeneProduct', 'None')
('is', ',')
('None', 'None')
('normally', 'is')
('None', 'None')
('found', 'normally')
('None', 'None')
('in', 'found')
('None', 'None')
('the', 'in')
('None', 'None')
('cells', 'the')
('None', 'None')
('that', 'cells')
('None', 'None')
('make', 'that')
('None', 'None')
('up', 'make')
('None', 'None')
('connective', 'up')
('None', 'None')
('tissues', 'connective')
('None', 'None')
('like', 'tissues')
('None', 'None')
('bone', 'like')
('None', 'None')
('and', 'bone')
('None', 'None')
('cartilage', 'and')
('None', 'None')
('.', 'cartilage')
('None', 'None')


In [294]:
full_ent_list[0]

['BBGeneOrGeneProduct',
 'IIGeneOrGeneProduct',
 'IIGeneOrGeneProduct',
 'IIGeneOrGeneProduct',
 'None',
 'None',
 'BBGeneOrGeneProduct',
 'IIGeneOrGeneProduct',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'BBGeneOrGeneProduct',
 'IIGeneOrGeneProduct',
 'None',
 'None',
 'None',
 'BBGeneOrGeneProduct',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'BBGeneOrGeneProduct',
 'IIGeneOrGeneProduct',
 'None',
 'None',
 'None'

In [296]:
def create_pred_list(res):
  pred_list2 = []
  for item in res:
    item = item.replace("BB", "BBBB")
    item = item.replace("II", "IIII")
    pred_list2.append(item)
  return pred_list2
predictions = create_pred_list(full_ent_list)

In [297]:
print(predictions)

['BBBBGeneOrGeneProduct', 'IIIIGeneOrGeneProduct', 'IIIIGeneOrGeneProduct', 'IIIIGeneOrGeneProduct', 'None', 'None', 'BBBBGeneOrGeneProduct', 'IIIIGeneOrGeneProduct', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'BBBBGeneOrGeneProduct', 'IIIIGeneOrGeneProduct', 'None', 'None', 'None', 'BBBBGeneOrGeneProduct', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'BBBBGeneOrGeneProduct', 'IIIIGeneOrGeneProduct', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'Non

In [301]:
actual_labels = sum(list(testing_set['ner_tags_labels']), [])

In [302]:
print(actual_labels)

['BBBBGeneOrGeneProduct', 'IIIIGeneOrGeneProduct', 'IIIIGeneOrGeneProduct', 'IIIIGeneOrGeneProduct', 'None', 'None', 'BBBBGeneOrGeneProduct', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'BBBBCellLine', 'None', 'None', 'None', 'None', 'None', 'BBBBGeneOrGeneProduct', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '

In [303]:
predictions2 = [label2id[x] for x in predictions]
actual_labels2 = [label2id[x] for x in actual_labels]

In [300]:
print(predictions2)

[1, 2, 2, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 3, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 

In [304]:
print(actual_labels2)

[1, 2, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [307]:
import evaluate
metric = evaluate.load('f1')
metric.compute(predictions = predictions2, references = actual_labels2, average = 'macro')

ValueError: ignored